In [3]:
import pandas as pd

In [10]:
# I will try to improve the model using an evenly dustributed technique
df = pd.read_csv("New_London.csv",encoding= 'unicode_escape')
df.head()

,Unnamed: 0,Property Name,Review Rating,Review Title,Review Text,Location Of The Reviewer,Date Of Review
0,0,Apex London Wall Hotel,5.0,Ottima qualitÃ prezzo,Siamo stati a Londra per un week end ed abbiam...,"Casale Monferrato, Italy",10/20/2012
1,1,Corinthia Hotel London,5.0,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...,"Savannah, Georgia",3/23/2016
2,2,The Savoy,5.0,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...,London,7/30/2013
3,3,Rhodes Hotel,4.0,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...,"Maui, Hawaii",06/02/2012
4,4,The Savoy,5.0,Perfection,Form the moment we arrived until we left we ex...,"London, United Kingdom",11/24/2017


In [54]:
import random

class Sentiment:
    positive = "Positive"
    negative = "Negative"
    neural = "Neural"
    


class Review:
    def __init__(self, text, rating):
        self.text = text
        self.rating = rating
        self.sentiment = self.get_rating()
        
    def get_rating(self):
        if self.rating > 4:
            return Sentiment.positive
        elif self.rating >= 3 and self.rating <= 4 :
            return Sentiment.neural
        else:
            return Sentiment.negative

# I will try to improve the model to improve the prediction for positive and negative test        
class Review_text:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
        
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.negative, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.positive, self.reviews))
        positive_shrink = positive[:len(negative)]
        self.reviews = negative + positive_shrink
        random.shuffle(self.reviews)
#         print(negative[0].text)
#         print(len(negative))
#         print(len(positive))
        

In [55]:
reviews = []
for index,item in df.iterrows():
    reviews.append(Review(item["Review Text"], item["Review Rating"]))
    
# Chech some data
reviews[0].rating
#reviews[0].text  


5.0

In [56]:
# Split the data to train and test the model
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_review = Review_text(training)
test_review = Review_text(test)



In [57]:
print(len(training))
len((test))

15662


7715

In [73]:
# I want to create an evenly distribute between positives and negatives commnets 
train_review.evenly_distribute()

train_x = train_review.get_text()
train_y = train_review.get_sentiment()

# To solve the issue that I had I will apply also evenly distribute to the data test
test_review.evenly_distribute()

test_x = test_review.get_text()
test_y = test_review.get_sentiment()

print(train_y.count(Sentiment.positive))
train_y.count((Sentiment.negative))

722


722

In [87]:
# I want to conver the text to numeric type
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# I can improve the model by changing this method (check)
# vectorizer = CountVectorizer()
vectorizer = vectorizer = TfidfVectorizer()    # Applying this method I have increaded the model accurate

# fit and transform 

train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x_vectors[0].toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [88]:
# I made the classification using the SVM algorthim

from sklearn import svm

clf_svm = svm.SVC(kernel="linear")

clf_svm.fit(train_x_vectors, train_y)

test_x[0]
#text_x_vectors[0]

# we can predict if this text is positive or negative

clf_svm.predict(test_x_vectors[1:10])

array(['Positive', 'Negative', 'Negative', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Negative'], dtype='<U8')

In [89]:
# Evaluation
## Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))

0.9299719887955182


In [90]:
# F1 Scores 
from sklearn.metrics import f1_score

print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=(Sentiment.positive, Sentiment.negative)))


[0.92857143 0.93131868]


In [91]:
# I had a big difference in the data testing before I apply the distribute method  
print(test_y.count(Sentiment.negative))
test_y.count((Sentiment.positive))

357


357

In [86]:
# Quality analysis 

test_set = ["Great place", "The most beautiful room", "It was a very bad hotel", "I don not recommend this place"]
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['Positive', 'Positive', 'Negative', 'Negative'], dtype='<U8')

In [ ]:
## Save the model

In [102]:
import pickle 

In [107]:
with open("./Data/sentiment_classifier.pkl", "wb") as f:
     pickle.dump(clf_svm,f)

In [108]:
## Load the model
with open("./Data/sentiment_classifier.pkl", "rb") as f:
    loaded_clf = pickle.load(f)

In [112]:
# For instance
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])

We recently stayed at The Savoy for our 5 year anniversary, and we couldn't have been more impressed with the experience. We were greeted by our charming and friendly host Johannes, and informed that we had been upgraded to a riverside suite. | The view down the river was magnificent, overlooking major city landmarks. | As an unexpected bonus, Johannes treated us to a tour of some of the major suites, packed with lots of fascinating historical tit bits about the hotel. | We ate in Kasper's restaurant and can highly recommend doing so. As a vegetarian, I was very pleased to find they have a separate vegetarian menu, with plenty of options, and my partner was delighted with his choice of fish dishes. | The butler service breakfast bought to our suite was the perfect way to round of our stay. Formed of a delicious spread of Eggs Royale, waffles and pastries, it was luxurious end to a very memorable experience.


array(['Positive'], dtype='<U8')